In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1 File input and output
One of the most important tools in Python is the ability to read files from your computer and write new files or update existing files. I am going to go through a few simple examples of how to read and write data using Python.
## 1.1 Reading a file
Reading a file requires the opening of a file handle and then use of one of the several file read methods:
### 1.1.1 The read() method
This method will read the entire contents of the filehandle into a variable. You can limit the amount read by feeding the method a number as an argument. Consider the following example, which will read the first 11 bytes from the file:

In [ ]:
edgar=open('company20194.idx','r')
print(edgar.read(1000))
edgar.close()

### 1.1.2 The readline() method
You can read also read entire lines of the file using **readline()**. Each time the method is invoked, it will return a subsequent line from the file

In [ ]:
edgar=open('company20194.idx','r')
print("Line 1: ", edgar.readline())
print("Line 2: ", edgar.readline())
print("Line 3: ", edgar.readline())
edgar.close()

### 1.1.3 The readlines() method
You can also read the lines of the file into a list such that each element of the list contains a different line.

In [ ]:
edgar=open('company20194.idx','r')
x=edgar.readlines()
print(len(x))
for line in range(10):
    print(x[line])
edgar.close()

### 1.1.4 Reading lines with a for loop
You can also read lines one-by-one withe a **for** loop. Notice the **enumerate** function that is used in the loop declaration. This is a convenient way to create a line counter.

In [ ]:
edgar=open('company20194.idx','r')
for (c, line) in enumerate(edgar,1):
    if c<11:
        print(line)
edgar.close()

### 1.1.5 Using the with statement to avoid the need to close filehandles
When you open a filehandle, it needs to be closed. This can be somewhat annoying. A covenient workaround is the use of the **with** statement.

In [ ]:
with open('company20194.idx','r') as edgar:
    for (c, line) in enumerate(edgar,1):
        if c<11:
            print(line)

## 1.2 Writing to a file
Writing text to a file is very similar to reading a file. One difference is that when you open the filehandle for writing, you will need to specify either 'w' or 'a' instead of 'r' depending on whether you would like to overwrite an existing file or append to a file if it exists.

In [ ]:
with open('output.txt', 'w') as fileout:
    fileout.write('This is how we write to a file.\n')
    
with open('output.txt', 'a') as f2:
    f2.write('Now we are appending another line to the file.\n')

# 2 Getting WRDS data with Python
Since the WRDS Cloud now houses its datasets in PostgreSQL format rather than a pile of .sas7bdat files, it is very convenient to pull some data from WRDS for either analysis or as part of some other task that you might have in mind. We will spend just a few minutes with the **wrds** module before moving into some more details about **pandas** and ways of looking at numerical data within Python. I encourage you to play around with Python and data analysis. See if you can use it for any tasks that might have been done in SAS or Stata. 

In [ ]:
import wrds
db = wrds.Connection(wrds_username='sbonsall')


#The step below creates a file on your computer so that you do not 
#need to enter your password in the future when using the WRDS
#module. Quite convenient!

#db.create_pgpass_file()

In [ ]:
# Listing the WRDS libraries
db.list_libraries()

In [ ]:
# Listing the tables within a 
db.list_tables('rpna')

In [ ]:
# You can also describe the contents of a table

db.describe_table(library='comp', table='adsprate')

In [ ]:
#There are two ways of querying data from WRDS. 
#The first is get_table and the second is raw_sql.

## get_table

ratings=db.get_table(library='comp',table='adsprate', columns=['gvkey','datadate','splticrm'])



In [ ]:
ratings[ratings['gvkey']=='001690']

In [ ]:
## raw_sql

prices=db.raw_sql("select permno, date, ret from crsp.dsf where date_part('year',date)=2019 limit 100")
prices

In [ ]:
prices2=db.raw_sql("select permno, date, ret from crsp.dsf where date_part('year',date)=2019")  
db.close()

# 3 Working with tabular data: The Pandas package
Text analysis typically involves unstructured and structured data. Un- structured data is data that is not organized in a pre-defined manner, such as a 10-K or an image. A 10-K is unstructured since each company organizes its 10-K differently. Much of this paper deals with unstruc- tured text data. However, researchers performing text analysis will also work with structured data such as SAS and Stata datasets, Excel files, and CSV files corresponding to Compustat, IBES, and CRSP data. Structured data is organized into pre-defined columns, each of which contains a specific type of data, and rows, each of which contains an observation.
## 3.1 Required import statements
The most common method of importing the Pandas library is with this call to the **import** statement. It is common practice to use the alias pd for Pandas, and we recommend the reader to do the same.

In [ ]:
import pandas as pd

The Pandas library is built on top of a numerical analysis library called NumPy, and most Pandas users frequently call NumPy functions to manipulate data. Therefore, most users include the following import statements at the top of their notebooks. We recommend the reader to do the same.

In [ ]:
import numpy as np

## 3.2 Importing and exporting data
In this section, we introduce Pandas’ capabilities for importing data from Excel, CSV, Stata, and SAS formats. We provide sample code and explanation for common use cases. Pandas provides powerful and flexible import capabilities and supports many other data formats. Consult the [official documentation](https://pandas.pydata.org/docs/user_guide/io.html) for use cases not covered here.

All Pandas functions for importing data begin with **read_**, e.g. **pd.read_excel**, **pd.read_sas**, etc. The options for these functions are fairly standardized so the learning curve flattens after the user gains proficiency with one import function.
### 3.2.1 Importing data from Excel
Use the function **pd.read_excel** to import data from an Excel file. If the Excel file contains only one worksheet with well-formatted data (i.e., one contiguous block of data with header row and no blank lines at the top of the worksheet) then **pd.read_excel** requires only one argument, the path to the file.

In [ ]:
import numpy as np
import pandas as pd

df=pd.read_excel('Categorical_EPU_Data.xlsx')

Notice that the pd.read_excel function assumed that the first row contained column names and automatically used them as column labels in the DataFrame. You can preview the DataFram by typing its name (**df**) into a code cell.

In [ ]:
df

One word of caution: attempting to use **pd.read_excel** on a file that is open in Excel will throw an error. To be able to read the file in Pandas, we need to close the file in Excel and try again.
#### Using pd.read_excel with an Excel Workbook Containing Multiple Worksheets
Excel workbooks often contain multiple worksheets. By default, the function **pd.read_excel** will read data from the first worksheet in the workbook. To read a different worksheet, use the optional keyword argument sheet_name. This argument accepts the following values:
- A case-sensitive string containing the name of the worksheet.
- A zero-based integer index of the worksheet. 0 means the first
worksheet, 1 means the second worksheet, and so on.

#### Using pd.read_excel When the Header Row is Missing
By default, **read_excel** assumes that the first row of data contains column **names**. If these column names are missing, pass the keyword argument **header=None**. Provide column names by passing a list to the optional keyword argument names.
#### Skipping Rows and Blank Lines
Many Excel users place blank lines and text above the data in a worksheet. To handle this use case, use the optional keyword argument skiprows when calling **pd.read_excel**. To skip the first n lines of the file, use **skiprows=n**. Alternatively, to skip specific rows, pass a list of row numbers; unlike Excel, skiprows assumes row numbers begin at zero.

In [ ]:
df2=pd.read_excel('Categorical_EPU_Data.xlsx', header=None, skiprows=1, names=['date','epu','monetary_policy','fiscal_policy','taxes','gov_spending','health_care','national_security','entitlement_programs','regulation','financial_regulation','trade_policy','sov_debt_curr_crises'])
df2.head()

#### Getting Help for pd.read_excel
The official documentation page for pd.read_excel is located here. Alternatively, type **help(pd.read_excel)** into a Jupyter notebook cell and execute the cell.
### 3.2.2 Importing data from a CSV file
CSV stands for “comma separated value.” A CSV file is a text file that uses commas to separate, or delimit, values. CSV format is commonly used to store small-to-medium sized data files because it can be used to transfer data between different software (e.g., SAS and Stata), and because all major operating systems support it.

By convention, the first row of a CSV file contains the column names. The Pandas function **pd.read_csv** is similar to **pd.read_excel**. The two functions accept many of the same arguments, such as **skiprows** and **names**. However, since CSV files are a text files. Some issues commonly arise when importing financial data in CSV format. In the remainder of this section, we demonstrate how to handle two of these issues: parsing date columns.

In [ ]:
import numpy as np
import pandas as pd

df3=pd.read_csv('EarningsNews.csv')
df3.head()

In [ ]:
df3.info()

#### Parsing Dates
The **pd.read_csv** function can parse dates. Simply tell the function which columns contain dates through the parse_dates keyword ar- gument, and Pandas usually imports the dates correctly.

To import the *date* field from our CSV file, we can use the following modification to our code:

In [ ]:
df4=pd.read_csv('EarningsNews.csv', parse_dates=['date'], dayfirst=True)
df4.info()
df4.head()

#### Getting Help for pd.read_csv
The official documentation page for **pd.read_csv** is located [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). Alternatively, type **help(pd.read_csv)** into a Jupyter notebook cell and execute the cell.
### 3.2.3 Importing Data from Stata
Use the function **pd.read_stata** to read Stata .dta files. At the time of this writing, **pd.read_stata** supports Stata versions 10–14. If your Stata file was written by a later version of Stata, either save it to an earlier version or save it to Excel or CSV.
### 3.2.4 Importing Data from SAS
Use the function **pd.read_sas** to read SAS files. This function can read SAS xport (.XPT) files and SAS7BDAT files. In our experience, this function can be finicky. If you have a valid SAS license and SAS installation on your computer and plan to regularly pass data between SAS and Python, we highly recommend the package **SASPy**. This package was written by The SAS Institute and is officially supported.
### 3.2.5 Exporting Data
Every **read_** function discussed in this paper has a corresponding **write_** function. It is therefore straightforward to save data to a desired format.
## 3.3 Viewing Data in Pandas
Researchers working with Pandas in Jupyter Notebooks have several options for viewing data. To view an entire DataFrame (or Series), simply type the name of the variable into a cell and run the cell. The data will be shown underneath the cell. If the DataFrame has many rows, Jupyter will show the top and bottom rows. Pandas users typically only need to view a few rows at a time. This is accomplished using the head and tail commands. For example, if the variable df stores a DataFrame, **df.head()** will display the first 5 rows of that DataFrame, and **df.tail()** will display the last 5 rows. To specify the number of rows to display, pass an optional argument to head or tail. Thus, **df.head(3)** will display the first 3 rows of *df*.

Use DataFrame’s sample method to view randomly selected rows. **df.sample()** will display one randomly chosen row and **df.sample(n)** will display n randomly chosen rows.
## 3.4 Selecting and Filtering Data

This chapter demonstrates the basics of selecting and filtering data in Pandas DataFrames. By selecting, we mean choosing a subset (usually columns) of a DataFrame. By filtering, we mean choosing a subset of rows with values that meet certain criteria. Pandas provides many features for selecting and filtering data and there are often multiple ways to do the same thing. We will only scratch the surface of Pandas’ capabilities.
### 3.4.1 Selecting Columns of a DataFrame
To select a subset of columns, you can call the frame with a list argument inside square brackets:

In [ ]:
ratings.head()
ratings[['gvkey','datadate']]

### 3.4.2 Filtering a DataFrame
Filtering is the process of extracting rows of a dataset that meet certain criteria. The main idea behind filtering in Pandas is to pass a Boolean series to a DataFrame; only rows for which the Boolean Series is true are kept in the filtered DataFrame.

In [ ]:
ratings['datadate']=pd.to_datetime(ratings['datadate'],format='%Y-%m-%d')
ratings_00s=ratings[(ratings['datadate'].dt.year>=2000) & (ratings['datadate'].dt.year<=2009)].copy()
ratings_00s.sort_values('datadate')

## 3.5 Creating new columns
Each column of a DataFrame is a Series. Therefore, creating a new column implies creating a Series. Typically, new columns are created as transformations of existing columns. However, sometimes new columns contain single values.
### 3.5.1 Creating a column from a scalar value
Pandas makes it very easy to create a new column that contains a single, repeated value. Simply assign that value to a column of the DataFrame that does not already exist.

In [ ]:
ratings['newcol']=1
ratings

This code tells Pandas to first create a DataFrame containing a single column. The second line of code references a column named 'newcol' in DataFrame *ratings*. Since that column does not exist, Pandas will create it. When Pandas sees a single, scalar value assigned to a Series, it automatically adds that value to every row of the DataFrame.

To summarize, to create a new column in a DataFrame containing
a single, repeated value:

- Type the DataFrame name followed by square brackets.

- Inside the square brackets, type the name of the new column as a string.

- After the brackets, type an equals sign and then the desired value.

### 3.5.2 Creating a column as a transformation of an existing column
Column transformations are very common. Examples of column trans- formation are to (1) strip whitespace from all values in a column, (2) convert a string column to uppercase, and (3) perform a mathematical operation on every value in a column. In this section, we provide sample code that implements these examples. Often, when cleaning data like this, it is desirable to do so “in place” (i.e., instead of creating a new column, replace data in an existing column with transformed data).

We will extract the year from 'datadate' in the *ratings* DataFrame and add it to the DataFrame:

In [ ]:
ratings['year']=ratings['datadate'].dt.year
ratings

## 3.6 Dropping and renaming columns
### 3.6.1 Dropping columns
Use the aptly named drop method to drop columns from a Pandas DataFrame. Pass a list of unwanted column names to the columns keyword argument. For example, if we wanted to drop the 'newcol' column from the *ratings* DataFrame, we could do the following:

In [ ]:
ratings.drop(columns=['newcol'], inplace=True)
ratings

Notice the keyword argument inplace. By default, the drop method returns a new DataFrame. To force drop to operate in place, this argument is needed. An alternative method for dropping columns in place is to reassign the result to the existing DataFrame. For example:

In [ ]:
ratings2=ratings.drop(columns=['year'])
ratings2
ratings

### 3.6.2 Renaming columns
The **rename** method works similarly to the **drop** method demonstrated above. However, instead of passing a list to the **columns** keyword argument, rename requires a *dictionary*. Each key in the dictionary must be the name of an existing column; the corresponding value is the new name for that column. This should be clear from the following example.

We will change the names of the 'datadate' and 'splticrm' variables in the ratings DataFrame:

In [ ]:
ratings.rename(columns={'datadate':'rating_date','splticrm':'rating'}, inplace=True)
ratings

## 3.7 Sorting data
To sort data in a DataFrm, use the **sort_values** method. Use the following keyword arguments to achieve the desired result:

- **by**: pass a list of column names by which to sort

- **ascending**: Use **True** (**False**) to sort all columns in ascending (descending) order. Alternatively, pass a list of Booleans to specify the sort order for each column named in the by argument.

- **inplace**: Use **True** to sort in-place. If this argument is omitted or set to **False**, then **sort_values** will return a new, sorted DataFrame.

Let's sort the *ratings* DataFrame by year and rating.

In [ ]:
sorted_ratings=ratings.sort_values(by=['year','rating'], ascending=[True,False])
sorted_ratings[(sorted_ratings['year']==2000) & (sorted_ratings['rating'].notna())]

## 3.8 Summarizing data
You can easily use the Pandas package to get summary statistics for your data. Summary statistics can be applied using various methods and can be applied to groups of variables. For instance, we can collapse the EPU data that we imported earlier to the annual level.

In [ ]:
df2['year']=df2['date'].dt.year
df2
## Only retain 'EPU'
annual_epu=df2.groupby('year')['epu'].mean()
annual_epu

## Retain all variables at the annual level
annual=df2.groupby('year').mean()
annual

## 3.9 Merging data
Just like in Stata, SAS, and SQL, the Pandas module in Python can merge different DataFrames and Series together in a database-style approach.

Let's try to merge the credit rating data with a few variables from the Compustat annual file.

In [ ]:
db=wrds.Connection(wrds_username='sbonsall')

comp=db.raw_sql("select gvkey, datadate, at from comp.funda where date_part('year',datadate) between 2000 and 2010 and indfmt='INDL' and consol='C' and datafmt='STD'")

db.close()

In [ ]:
comp['datadate']=pd.to_datetime(comp['datadate'],format='%Y-%m-%d')
comp.info()
ratings.info()
merged=pd.merge(comp,ratings, left_on=['gvkey','datadate'], right_on=['gvkey','rating_date'],indicator=True, validate='1:1')
merged

## 3.10 Visualization
You can do a lot with graphs in Pandas and Python. The main module for plotting is called **matplotlib**. We will load that and then see how to graph. Obviously, you will have to play around with plotting for a while to a hang of all the available features.

In [ ]:
import matplotlib.pyplot as plt
df.info()
df.plot(x='Date')
plt.ylabel('Index Level')
plt.legend(loc='right', bbox_to_anchor=(1.7, 0.5))
plt.title('Policy Uncertainty Over Time')
plt.savefig('epu.png',dpi=400, bbox_inches='tight')